# **Load data for stations and edges, construct the undirected graph**

In [2]:
#put the stations.txt and edges.txt in the same directory with this notebook
#change the path to your data path

#Load stations.txt
dic_station = {}
f = open("/content/drive/My Drive/STEP/stations.txt")
lines = f.readlines()
for line in lines:
  station_id = line.split('\t',2)[0]
  station_name = line.split('\t',2)[1].split('\n',2)[0]
  dic_station[station_id] = station_name
f.close()

#Load edges.txt
graph = {}
f = open("/content/drive/My Drive/STEP/edges.txt")
lines = f.readlines()
for line in lines:
  id_1 = line.split('\t',3)[0]
  id_2 = line.split('\t',3)[1]
  time = int(line.split('\t',3)[2].split('\n',2)[0])
  if dic_station[id_1] not in graph.keys():
    graph[dic_station[id_1]] = {dic_station[id_2]: time}
  else:
    if dic_station[id_2] not in graph[dic_station[id_1]].keys():
      graph[dic_station[id_1]][dic_station[id_2]] = time
  if dic_station[id_2] not in graph.keys():
    graph[dic_station[id_2]] = {dic_station[id_1]: time}
  else:
    if dic_station[id_1] not in graph[dic_station[id_2]].keys():
      graph[dic_station[id_2]][dic_station[id_1]] = time    
f.close()

graph['高田馬場'] 

{'新大久保': 2, '早稲田': 3, '目白': 3, '落合': 3}

# **Route guidance**

In [9]:
#Find the shortest path (shortest time) between station 1 and station 2 using Dijkstra’s algorithm
departure = '駒込'
destination = '渋谷'
reserved = departure

#keep track of the visited and unvisited stations
visited = set()
unvisited = set([key for key in graph.keys()])
#store the dijkstra information in a table (i.e. including the shortest distance from the departure station and the previous vertex)
table = {}
for key in graph.keys():
  if key == departure:
    table[key] = [0,'']
  else:
    table[key] = [float('inf'),'']

#loop through all unvisited stations
while unvisited:
  for neighbour in graph[departure]:
    if neighbour not in visited:
      #calculate the new cost of the new path, if it's smaller than the stored one, update it
      new_cost = table[departure][0] + graph[departure][neighbour]
      if new_cost < table[neighbour][0]:
        table[neighbour][0] = new_cost
        table[neighbour][1] = departure
  visited.add(departure)
  unvisited.remove(departure)
  next_departure = ''
  temp = float('inf')
  #visited the unvisited vertex with the smallest known distance from the start vertex
  for station in unvisited:
    if table[station][0] < temp:
      temp = table[station][0]
      next_departure = station
  departure = next_departure


departure = reserved
path = [destination]
time = []
#back tracking from destination to departure station and record the stations on the　path
while destination != departure:
  path.append(table[destination][1])
  time.append(table[destination][0])
  destination = table[destination][1]

path.reverse()
time.append(0)
time.reverse()
print('======================================================')
print('The recommended path is:')
print(path)
print('======================================================')
print('The total time you need to spend on the way:')
print(time[-1], 'mins')
print('======================================================')
print('Detailed time information:')
for i in range(len(path)-1):
  print(path[i], ' -> ', path[i+1], '\t', time[i+1]-time[i], 'mins', '\ttotal time cost from departure station:', time[i+1], 'mins')


The recommended path is:
['駒込', '本駒込', '東大前', '後楽園', '飯田橋', '九段下', '半蔵門', '永田町', '青山一丁目', '表参道', '渋谷']
The total time you need to spend on the way:
22 mins
Detailed time information:
駒込  ->  本駒込 	 2 mins 	total time cost from departure station: 2 mins
本駒込  ->  東大前 	 2 mins 	total time cost from departure station: 4 mins
東大前  ->  後楽園 	 3 mins 	total time cost from departure station: 7 mins
後楽園  ->  飯田橋 	 2 mins 	total time cost from departure station: 9 mins
飯田橋  ->  九段下 	 2 mins 	total time cost from departure station: 11 mins
九段下  ->  半蔵門 	 2 mins 	total time cost from departure station: 13 mins
半蔵門  ->  永田町 	 2 mins 	total time cost from departure station: 15 mins
永田町  ->  青山一丁目 	 3 mins 	total time cost from departure station: 18 mins
青山一丁目  ->  表参道 	 2 mins 	total time cost from departure station: 20 mins
表参道  ->  渋谷 	 2 mins 	total time cost from departure station: 22 mins


In [10]:
#If not weights are taken into consideration, just find the shortest path
from collections import deque

#Find shortest path from one node to another
def bfs(visited_node, graph, current_node, end_node):
  #store all the paths
  queue = deque([[current_node]])
  while len(queue) > 0:
    current_path = queue.popleft()
    current_node = current_path[-1]
    if current_node not in visited_node:
      neighbours = graph[current_node].keys()
      for neighbour in neighbours:
        #form new path
        temp = current_path.copy()
        temp.append(neighbour)
        if neighbour == end_node:
          return temp
        queue.append(temp)
  return None
    

station_1 = '駒込'
station_2 = '渋谷'
visited_node = []
visited_node = bfs(visited_node, graph, station_1, station_2)
print(visited_node)

['駒込', '巣鴨', '大塚', '池袋', '目白', '高田馬場', '新大久保', '新宿', '代々木', '原宿', '渋谷']


In [8]:
#If weights are taken into consideration
#But not better than dijkstra algorithm, i think

from collections import deque
def bfs(visited_node, graph, current_node, end_node):
  #store all the paths
  time = deque([[0]])
  min_time = float('inf')
  queue = deque([[current_node]])
  path = []
  path_time = []
  while len(queue) > 0:
    current_path = queue.popleft()
    current_node = current_path[-1]
    current_time = time.popleft()
    if current_node not in visited_node:
      neighbours = graph[current_node].keys()
      for neighbour in neighbours:
        #form new path
        temp = current_path.copy()
        temp2 = current_time.copy()
        temp.append(neighbour)
        temp2.append(graph[current_node][neighbour])
        queue.append(temp)
        time.append(temp2)
        if neighbour == end_node:
          if sum(temp2)<=min_time:
            min_time=sum(temp2)
            print(queue[-1])
            print(sum(time[-1]),'mins')
            path.append(queue[-1])
            path_time.append(time[-1])
          else:
            return 'Finished searching'

    

station_1 = '駒込'
station_2 = '渋谷'
visited_node = []
visited_node = bfs(visited_node, graph, station_1, station_2)
print(visited_node)

['駒込', '巣鴨', '大塚', '池袋', '目白', '高田馬場', '新大久保', '新宿', '代々木', '原宿', '渋谷']
24 mins
['駒込', '巣鴨', '大塚', '池袋', '雑司が谷', '西早稲田', '東新宿', '新宿三丁目', '北参道', '明治神宮前〈原宿〉', '渋谷']
24 mins
['駒込', '本駒込', '東大前', '後楽園', '飯田橋', '市ケ谷', '四ツ谷', '赤坂見附', '青山一丁目', '表参道', '渋谷']
22 mins
['駒込', '本駒込', '東大前', '後楽園', '飯田橋', '市ケ谷', '四ツ谷', '永田町', '青山一丁目', '表参道', '渋谷']
22 mins
['駒込', '本駒込', '東大前', '後楽園', '飯田橋', '市ケ谷', '麹町', '永田町', '青山一丁目', '表参道', '渋谷']
22 mins
['駒込', '本駒込', '東大前', '後楽園', '飯田橋', '九段下', '半蔵門', '永田町', '青山一丁目', '表参道', '渋谷']
22 mins
Finished searching
